# OMS Geofabrik

OpenStreetMap (OSM) is a collaborative project that creates a free, editable map of the world, used widely for geospatial analysis and mapping projects.

Geofabrik is a well-known provider that offers regularly updated regional extracts of OSM data in convenient file formats like .osm.pbf (a compressed binary format). 

## Setup

Package install from terminal (if required)

```
conda install -c conda-forge pyrosm
conda install -c conda-forge osmnx
```

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import os
import osmnx as ox
from pyrosm import OSM
import geopandas as gpd

In [9]:
def get_latest_geofabrik_china_url():
    base_url = "https://download.geofabrik.de/asia"
    response = requests.get(base_url + "/china.html")
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find the .osm.pbf link
    links = soup.find_all('a', href=True)
    pbf_links = [link['href'] for link in links if re.search(r'china(-latest)?\.osm\.pbf$', link['href'])]

    if not pbf_links:
        raise Exception("No .osm.pbf link for China found on the Geofabrik page.")
    
    # Build the full download URL
    return base_url +"/"+ pbf_links[0]

def download_file(url, dest_folder):
    os.makedirs(dest_folder, exist_ok=True)
    local_filename = url.split('/')[-1]
    local_path = os.path.join(dest_folder, local_filename)
    
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    return local_path

if __name__ == "__main__":
    print("Fetching latest China OSM PBF download link from Geofabrik...")
    latest_url = get_latest_geofabrik_china_url()
    print(f"Download link found: {latest_url}")
    
    print("Starting download to ../../data/mapdata ...")
    downloaded_file = download_file(latest_url, dest_folder="../../data/mapdata")
    print(f"File successfully downloaded to: {downloaded_file}")

Fetching latest China OSM PBF download link from Geofabrik...
Download link found: https://download.geofabrik.de/asia/china-latest.osm.pbf
Starting download to ../../data/mapdata ...
File successfully downloaded to: ../../data/mapdata\china-latest.osm.pbf


 ## Exploratory analysis